In [1]:
import torch
import torch.nn as nn
from d2l import torch as d2l

In [2]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    if not torch.is_autocast_enabled():
        X_hat = (X - moving_mean) / np.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            mean = X.mean(axis=0)
            var = ((X - mean) ** 2).mean(axis=0)
        else:
            mean = X.mean(axis=(0, 2, 3), keepdims=True)
            var = ((X - mean) ** 2).mean(axis=(0, 2, 3), keepdims=True)
        X_hat = (X - mean) / np.sqrt(var + eps)
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta # Scale and shift
    
    return Y, moving_mean, moving_var

In [ ]:
def BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims, **kwargs):
        super().__init__(**kwargs)
        
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        
        self.gamma = self.params.get("gamma", shape=shape, init=torch.FloatTensor(1))
        self.beta = self.params.get("beta", shape=shape, init=torch.FloatTensor(0))
        
        self.moving_mean = np.zeros(shape)
        self.moving_var = np.ones(shape)
        
    d